In [27]:
import os.path

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark


In [2]:
folder_path = '/Users/sg0218817/Downloads/Ex_Files_Spark_SQL_DataFrames/Exercise Files/Data'
df = spark.read.format('json')\
    .load(os.path.join(folder_path, 'utilization'))

df.show(10)
df.printSchema()
df.count()


+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|         event_date|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
+---------------+-------------------+-

500000

In [7]:
df.createOrReplaceTempView('utilization')

df_sql = spark.sql('SELECT server_id AS id, cpu_utilization AS cpu '
                   'FROM utilization '
                   'WHERE cpu_utilization > 0.5 '
                   'LIMIT 5')
df_sql.show()


+---+----+
| id| cpu|
+---+----+
|100|0.57|
|100|0.56|
|100|0.57|
|100|0.57|
|100|0.53|
+---+----+



In [17]:
df_sql = spark.sql('SELECT server_id AS id, ROUND(AVG(cpu_utilization), 4) AS avg_cpu '
                   'FROM utilization '
                   'GROUP BY server_id '
                   'ORDER BY avg_cpu DESC')
df_sql.show()


+---+-------+
| id|avg_cpu|
+---+-------+
|101| 0.7986|
|113| 0.7833|
|145| 0.7823|
|103| 0.7614|
|102| 0.7584|
|133| 0.7563|
|108| 0.7476|
|149| 0.7448|
|137| 0.7408|
|148| 0.7394|
|123| 0.7314|
|118| 0.7287|
|112| 0.7154|
|139| 0.7154|
|104| 0.7109|
|142| 0.7033|
|121|  0.702|
|146| 0.6997|
|126| 0.6785|
|144| 0.6739|
+---+-------+
only showing top 20 rows



In [52]:
df_server = spark.read.format('csv')\
    .option('header', 'true')\
    .load(os.path.join(folder_path, 'server_names.csv'))

df_server.createOrReplaceTempView('server_names')

df_sql = spark.sql('SELECT sn.server_name as `name`, AVG(u.cpu_utilization) as `avg_cpu` '
                   'FROM utilization u '
                   'INNER JOIN server_names sn ON u.server_id = sn.server_id '
                   'GROUP BY sn.server_name '
                   'ORDER BY avg_cpu DESC')
df_sql.show()


+-----------+------------------+
|       name|           avg_cpu|
+-----------+------------------+
|Server #101|0.7985559999999872|
|Server #113|0.7833319999999914|
|Server #145|0.7823149999999972|
|Server #103|0.7614389999999969|
|Server #102|0.7583949999999904|
|Server #133|0.7562959999999967|
|Server #108| 0.747636000000003|
|Server #149|0.7447700000000035|
|Server #137|0.7407720000000032|
|Server #148|0.7393840000000045|
|Server #123|0.7314100000000078|
|Server #118|0.7286740000000046|
|Server #112|0.7153870000000067|
|Server #139|0.7153610000000032|
|Server #104|0.7108530000000015|
|Server #142|0.7032940000000032|
|Server #121|0.7019790000000099|
|Server #146| 0.699687000000013|
|Server #126|0.6784700000000029|
|Server #144|0.6738750000000022|
+-----------+------------------+
only showing top 20 rows

